# In this notebook, we will be attempting to use transfer learning to predict our labels.

In [40]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16

from pathlib import Path
import imghdr

## Pre-process data

In [41]:
#  the data
data_dir = "./BBTrD/000000001111"
image_extensions = [".png", ".jpg", ".jpeg"]  # add there all your images file extensions

img_type_accepted_by_tf = ["bmp", "gif", "jpeg", "png"]
for filepath in Path(data_dir).rglob("*"):
    if filepath.suffix.lower() in image_extensions:
        img_type = imghdr.what(filepath)
        if img_type is None:
            print(f"{filepath} is not an image")
        elif img_type not in img_type_accepted_by_tf:
            print(f"{filepath} is a {img_type}, not accepted by TensorFlow")

## Load Images and Labels

In [42]:
def load_labels():
    txt_file_path = "labels.txt"
    labels = []
    with open(txt_file_path, 'r') as f:
        for line in f:
            labels.append(line.strip())
    return labels

In [43]:
game_states = load_labels()
game_states = sorted(game_states)
game_states

['000000001111',
 '000000011110',
 '000000101101',
 '000000111100',
 '000001001011',
 '000001011010',
 '000001101001',
 '000001111000',
 '000010000111',
 '000010010110',
 '000010100101',
 '000010110100',
 '000011000011',
 '000011010010',
 '000011100001',
 '000011110000',
 '000100001110',
 '000100101100',
 '000101001010',
 '000101101000',
 '000110000110',
 '000110100100',
 '000111000100',
 '000111100000',
 '001000001101',
 '001000011100',
 '001001001001',
 '001001011000',
 '001010000101',
 '001010010100',
 '001011000001',
 '001011010000',
 '001100001100',
 '001101001000',
 '001110000100',
 '001111000000',
 '010000001011',
 '010000011010',
 '010000101001',
 '010000111000',
 '010010000011',
 '010010010010',
 '010010100001',
 '010010110000',
 '010100001010',
 '010100101000',
 '010110000010',
 '010110100000',
 '011000001001',
 '011000011000',
 '011010000001',
 '011010010000',
 '011100001000',
 '011110000000',
 '100000000111',
 '100000010110',
 '100000100101',
 '100000110100',
 '100001000011

In [44]:
image_height = 128
image_width = 128
batch_size = 81

def load_data():
    # Load Train Data
    train_images = tf.keras.utils.image_dataset_from_directory("./BBTrD/", batch_size=batch_size, image_size=(image_height, image_width), shuffle=False)

    # Load Test Data
    test_images =tf.keras.utils.image_dataset_from_directory("./BBTeD/", batch_size=batch_size, image_size=(image_height, image_width), shuffle=False)
    return train_images, test_images

In [45]:
train_images, test_images = load_data()

Found 81 files belonging to 81 classes.
Found 34 files belonging to 34 classes.


In [46]:
train_images.class_names

['000000001111',
 '000000011110',
 '000000101101',
 '000000111100',
 '000001001011',
 '000001011010',
 '000001101001',
 '000001111000',
 '000010000111',
 '000010010110',
 '000010100101',
 '000010110100',
 '000011000011',
 '000011010010',
 '000011100001',
 '000011110000',
 '000100001110',
 '000100101100',
 '000101001010',
 '000101101000',
 '000110000110',
 '000110100100',
 '000111000100',
 '000111100000',
 '001000001101',
 '001000011100',
 '001001001001',
 '001001011000',
 '001010000101',
 '001010010100',
 '001011000001',
 '001011010000',
 '001100001100',
 '001101001000',
 '001110000100',
 '001111000000',
 '010000001011',
 '010000011010',
 '010000101001',
 '010000111000',
 '010010000011',
 '010010010010',
 '010010100001',
 '010010110000',
 '010100001010',
 '010100101000',
 '010110000010',
 '010110100000',
 '011000001001',
 '011000011000',
 '011010000001',
 '011010010000',
 '011100001000',
 '011110000000',
 '100000000111',
 '100000010110',
 '100000100101',
 '100000110100',
 '100001000011

## Load Pre-trained VGG model

In [47]:
#load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [48]:
num_classes = 81
num_epochs = 10
# we will be freezing the base model's layers in order to keep the weights in the optimal condition they were trained in
for layer in vgg_conv.layers:
    layer.trainable = False

# new classifier model, which consists of the old model's layers plus some new ones
extra_layers = vgg_conv.output
# extra_layers = keras.layers.Conv2D(32, (3, 3), activation='relu')(extra_layers)
extra_layers = keras.layers.Flatten()(extra_layers)
extra_layers = keras.layers.Dense(1024, activation='relu')(extra_layers)

predictions = keras.layers.Dense(num_classes, activation='softmax')(extra_layers)

# create the new model
new_model = keras.models.Model(inputs=vgg_conv.input, outputs=predictions)

# compile the model
new_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
new_model.fit(train_images, epochs=num_epochs)

# print(train_images)





Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 52.7202 - accuracy: 0.0247
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 58.8859 - accuracy: 0.0370
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 64.8935 - accuracy: 0.1235
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 59.9015 - accuracy: 0.1605
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 56.5342 - accuracy: 0.1975
Epoch 6/10
1/1 [==============================] - 3s 3s/step - loss: 50.0474 - accuracy: 0.3086
Epoch 7/10
1/1 [==============================] - 3s 3s/step - loss: 50.6192 - accuracy: 0.3457
Epoch 8/10
1/1 [==============================] - 3s 3s/step - loss: 47.7985 - accuracy: 0.5309
Epoch 9/10
1/1 [==============================] - 3s 3s/step - loss: 49.0251 - accuracy: 0.5185
Epoch 10/10
1/1 [==============================] - 3s 3s/step - loss: 51.2094 - accuracy: 0.5309


In [49]:
# testing dropout
# new classifier model, which consists of the old model's layers plus some new ones
extra_layers = vgg_conv.output
# extra_layers = keras.layers.Conv2D(32, (3, 3), activation='relu')(extra_layers)
extra_layers = keras.layers.Flatten()(extra_layers)
extra_layers = keras.layers.Dense(1024, activation='relu')(extra_layers)
extra_layers = keras.layers.Dropout(0.2)(extra_layers)

predictions = keras.layers.Dense(num_classes, activation='softmax')(extra_layers)

# create the new model
dropout_model = keras.models.Model(inputs=vgg_conv.input, outputs=predictions)

# compile the model
dropout_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
dropout_model.fit(train_images, epochs=num_epochs)

# print(train_images)

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 59.2527 - accuracy: 0.0370
Epoch 2/10
1/1 [==============================] - 3s 3s/step - loss: 59.8752 - accuracy: 0.0123
Epoch 3/10
1/1 [==============================] - 3s 3s/step - loss: 53.6311 - accuracy: 0.1358
Epoch 4/10
1/1 [==============================] - 3s 3s/step - loss: 51.5611 - accuracy: 0.1358
Epoch 5/10
1/1 [==============================] - 3s 3s/step - loss: 43.9137 - accuracy: 0.2963
Epoch 6/10
1/1 [==============================] - 3s 3s/step - loss: 38.2325 - accuracy: 0.3580
Epoch 7/10
1/1 [==============================] - 3s 3s/step - loss: 34.1511 - accuracy: 0.3580
Epoch 8/10
1/1 [==============================] - 3s 3s/step - loss: 26.0233 - accuracy: 0.5185
Epoch 9/10
1/1 [==============================] - 3s 3s/step - loss: 21.3180 - accuracy: 0.5432
Epoch 10/10
1/1 [==============================] - 3s 3s/step - loss: 17.2067 - accuracy: 0.6914


# Using data augmentation

## Resize & Rescale Images

In [51]:
resize_and_rescale = tf.keras.Sequential([
    keras.layers.Rescaling(1./255)
])

train_images = train_images.map(lambda x, y: (resize_and_rescale(x), y))
test_images = train_images.map(lambda x, y: (resize_and_rescale(x), y))

## Data Augmentation

In [52]:
data_flip = tf.keras.Sequential([
  keras.layers.RandomFlip("horizontal_and_vertical")
])

data_rotation = tf.keras.Sequential([
  keras.layers.RandomRotation(0.2)
])

data_contrast = tf.keras.Sequential([
  keras.layers.RandomContrast(0.2)
])

data_shift = tf.keras.Sequential([
  keras.layers.RandomTranslation(0.2, 0.2)
])

flipped_images = train_images.map(lambda x, y: (data_flip(x), y))
rotated_images = train_images.map(lambda x, y: (data_rotation(x), y))
contrast_images = train_images.map(lambda x, y: (data_contrast(x), y))
shifted_images = train_images.map(lambda x, y: (data_shift(x), y))

train_images = train_images.concatenate(flipped_images)
train_images = train_images.concatenate(rotated_images)
train_images = train_images.concatenate(contrast_images)
train_images = train_images.concatenate(shifted_images) 

In [53]:
# test normal model
new_model.fit(train_images, epochs=num_epochs)

Epoch 1/10
5/5 [==============================] - 15s 3s/step - loss: 5.0433 - accuracy: 0.1160
Epoch 2/10
5/5 [==============================] - 15s 3s/step - loss: 4.2238 - accuracy: 0.3062
Epoch 3/10
5/5 [==============================] - 15s 3s/step - loss: 3.4724 - accuracy: 0.4519
Epoch 4/10
5/5 [==============================] - 15s 3s/step - loss: 2.9041 - accuracy: 0.5728
Epoch 5/10
5/5 [==============================] - 15s 3s/step - loss: 2.4486 - accuracy: 0.6568
Epoch 6/10
5/5 [==============================] - 15s 3s/step - loss: 2.0662 - accuracy: 0.7111
Epoch 7/10
5/5 [==============================] - 16s 3s/step - loss: 1.7862 - accuracy: 0.7457
Epoch 8/10
5/5 [==============================] - 16s 3s/step - loss: 1.5879 - accuracy: 0.7333
Epoch 9/10
5/5 [==============================] - 15s 3s/step - loss: 1.2837 - accuracy: 0.8272
Epoch 10/10
5/5 [==============================] - 15s 3s/step - loss: 1.1123 - accuracy: 0.8346


In [54]:
# test dropout model
dropout_model.fit(train_images, epochs=num_epochs)

Epoch 1/10
5/5 [==============================] - 15s 3s/step - loss: 3.8028 - accuracy: 0.2000
Epoch 2/10
5/5 [==============================] - 15s 3s/step - loss: 3.4015 - accuracy: 0.3358
Epoch 3/10
5/5 [==============================] - 15s 3s/step - loss: 2.9399 - accuracy: 0.5062
Epoch 4/10
5/5 [==============================] - 16s 3s/step - loss: 2.6278 - accuracy: 0.5531
Epoch 5/10
5/5 [==============================] - 15s 3s/step - loss: 2.1989 - accuracy: 0.6494
Epoch 6/10
5/5 [==============================] - 15s 3s/step - loss: 1.8551 - accuracy: 0.6889
Epoch 7/10
5/5 [==============================] - 15s 3s/step - loss: 1.6584 - accuracy: 0.7210
Epoch 8/10
5/5 [==============================] - 15s 3s/step - loss: 1.4874 - accuracy: 0.7654
Epoch 9/10
5/5 [==============================] - 15s 3s/step - loss: 1.3314 - accuracy: 0.7358
Epoch 10/10
5/5 [==============================] - 15s 3s/step - loss: 1.1914 - accuracy: 0.7753
